#Mount Google Drive

In [ ]:
from google.colab import files
from google.colab import drive

#Mount Google Drive
drive.mount('/content/drive')

#Import Library

In [ ]:
import pandas as pd
import numpy as np

#Inisialisasi Lokasi CSV

In [ ]:
base_directory = "drive/MyDrive/ETL Data Warehouse/"

directory_salh = base_directory + "salh.csv"
directory_sald = base_directory + "sald.csv"
directory_inv = base_directory + "inv.csv"
directory_cust = base_directory + "cust.csv"
directory_area = base_directory + "area.csv"
directory_salesperson = base_directory + "salperson.csv"
directory_rfm = base_directory + "rfm_per_customer.csv"

#ETL "product_dim"

In [ ]:
#Reading the dataset
product_dim_df = pd.read_csv(directory_inv, ",")
etl_product_dim_df = product_dim_df

In [ ]:
#Removing unused columns
etl_product_dim_df.drop(['SPRICE', 'UCOST', 'WEIGHT', 'UPDDATE', 'UPDTIME'], axis=1, inplace=True)

In [ ]:
#Changing the column names to match the star schema
etl_product_dim_df.rename(columns = {"ITEMNO":"product_id", "ITEMNAME":"product_name"}, inplace=True)

In [ ]:
#Converting the dataframe to CSV
etl_product_dim_df.to_csv("product_dim.csv", index=False)
files.download("product_dim.csv")

#ETL "customer_dim"

In [ ]:
#Reading the dataset
customer_dim_df = pd.read_csv(directory_cust, ";")
etl_customer_dim_df = customer_dim_df

In [ ]:
#Removing unused columns
etl_customer_dim_df.drop(['CODE', 'AREACD', 'SALTYPE', 'ARBAL', 'ADDR1', 'ADDR2', 'ADDR3', 'PHONE', 'UPDDATE', 'UPDTIME'], axis=1, inplace=True)

In [ ]:
#Changing the column names to match the star schema
etl_customer_dim_df.rename(columns = {"CUSTNAME":"customer_name", "INACTIVE":"customer_status"}, inplace=True)

In [ ]:
#Changing the "customer_status" from False (boolean) to "Active" (string) and from True (boolean) to "Inactive" (string)
etl_customer_dim_df.loc[etl_customer_dim_df["customer_status"] == False, "customer_status"] = "Aktif"
etl_customer_dim_df.loc[etl_customer_dim_df["customer_status"] == True, "customer_status"] = "Tidak Aktif"

In [ ]:
#Creating surrogate key for the "customer_dim" table

#Creating an empty dataframe with a column named "customer_id"
surrogate_key_customer_dim_df = pd.DataFrame(columns = ["customer_id"])

#Creating a surrogate key and then load the surrogate key into the empty dataframe that was previously created
for i in range(1, len(etl_customer_dim_df)+1, 1):
  surrogate_key_customer_dim_df = surrogate_key_customer_dim_df.append({'customer_id' : i}, ignore_index = True)

#Merging the surrogate key into the "customer_dim" dataframe
etl_customer_dim_df["customer_id"] = surrogate_key_customer_dim_df

In [ ]:
#Sorting the columns to match the star schema
etl_customer_dim_df = etl_customer_dim_df[["customer_id","customer_name","customer_status"]]

In [ ]:
#Converting the dataframe to CSV
etl_customer_dim_df.to_csv("customer_dim.csv", index=False)
files.download("customer_dim.csv")

#ETL "date_dim"

In [ ]:
#Reading the dataset
date_dim_df = pd.read_csv(directory_salh)
etl_date_dim_df = date_dim_df

In [ ]:
#Removing unused columns
etl_date_dim_df.drop(['TRNO', 'CUSTCODE', 'STYPE', 'TOTAL', 'UPDDATE', 'UPDTIME'], axis=1, inplace=True)

In [ ]:
#Changing the column names to match the star schema
etl_date_dim_df.rename(columns = {"TRDATE":"transaction_date"}, inplace=True)

In [ ]:
#Sorting the dates in the table from the earliest date to the latest date
etl_date_dim_df.sort_values("transaction_date", inplace = True)

In [ ]:
#Dropping duplicate dates from the dataframe
etl_date_dim_df.drop_duplicates(inplace = True)

In [ ]:
#Resetting the dataframe index because the old dataframe index is not in order
etl_date_dim_df.reset_index(drop=True, inplace=True)

In [ ]:
#Creating surrogate key for the "date_dim" table

#Creating an empty dataframe with a column named "date_id"
surrogate_key_date_dim_df = pd.DataFrame(columns = ["date_id"])

#Creating a surrogate key and then load the surrogate key into the empty dataframe that was previously created
for i in range(1, len(etl_date_dim_df)+1, 1):
  surrogate_key_date_dim_df = surrogate_key_date_dim_df.append({'date_id' : i}, ignore_index = True)

#Merging the surrogate key into the "date_dim" dataframe
etl_date_dim_df["date_id"] = surrogate_key_date_dim_df

In [ ]:
#Sorting the columns to match the star schema
etl_date_dim_df = etl_date_dim_df[["date_id","transaction_date"]]

In [ ]:
#Creating year, quarter, month, week, and day columns
etl_date_dim_df["transaction_date"] = pd.to_datetime(etl_date_dim_df["transaction_date"])

etl_date_dim_df['year'] = etl_date_dim_df['transaction_date'].dt.year
etl_date_dim_df['quarter'] = etl_date_dim_df['transaction_date'].dt.quarter
etl_date_dim_df['month'] = etl_date_dim_df['transaction_date'].dt.month
etl_date_dim_df['week'] = etl_date_dim_df['transaction_date'].dt.week
etl_date_dim_df['day'] = etl_date_dim_df['transaction_date'].dt.dayofyear

In [ ]:
#Converting the dataframe to CSV
etl_date_dim_df.to_csv("date_dim.csv", index=False)
files.download("date_dim.csv")

#ETL "area_dim"

In [ ]:
#Reading the dataset
area_dim_df = pd.read_csv(directory_area, ",")
etl_area_dim_df = area_dim_df

In [ ]:
#Removing unused columns
etl_area_dim_df.drop(['CODE', 'SALPERSON', 'UPDDATE', 'UPDTIME'], axis=1, inplace=True)

In [ ]:
#Changing the column names to match the star schema
etl_area_dim_df.rename(columns = {"DESC":"area_name"}, inplace=True)

In [ ]:
#Creating surrogate key for the "area_dim" table

#Creating an empty dataframe with a column named "area_id"
surrogate_key_area_dim_df = pd.DataFrame(columns = ["area_id"])

#Creating a surrogate key and then load the surrogate key into the empty dataframe that was previously created
for i in range(1, len(etl_area_dim_df)+1, 1):
  surrogate_key_area_dim_df = surrogate_key_area_dim_df.append({'area_id' : i}, ignore_index = True)

#Merging the surrogate key into the "area_dim" dataframe 
etl_area_dim_df["area_id"] = surrogate_key_area_dim_df

In [ ]:
#Sorting the columns to match the star schema
etl_area_dim_df = etl_area_dim_df[["area_id","area_name"]]

In [ ]:
#Adding the "latitude" column
latitude = [-6.598122, -6.715510, -6.705964, -6.396818, -6.569738, -7.443695, -6.937865, -6.606161]
etl_area_dim_df["latitude"] = latitude

In [ ]:
#Adding the "longitude" column
longitude = [106.802569, 107.014857, 107.753022, 106.81105, 106.628599, 107.094473, 106.926579, 106.802256]
etl_area_dim_df["longitude"] = longitude

In [ ]:
#Converting the dataframe to CSV
etl_area_dim_df.to_csv("area_dim.csv", index=False)
files.download("area_dim.csv")

#ETL "sales_person_dim"

In [ ]:
#Reading the dataset
sales_person_dim_df = pd.read_csv(directory_salesperson, ",")
etl_sales_person_dim_df = sales_person_dim_df

In [ ]:
#Removing unused columns
etl_sales_person_dim_df.drop(['DOB', 'GENDER', 'SALARY', 'UPDDATE', 'UPDTIME'], axis=1, inplace=True)

In [ ]:
#Changing the column names to match the star schema
etl_sales_person_dim_df.rename(columns = {"SALPCODE":"sales_person_id", "SALPNAME":"sales_person_name"}, inplace=True)

In [ ]:
#Converting the dataframe to CSV
etl_sales_person_dim_df.to_csv("sales_person_dim.csv", index=False)
files.download("sales_person_dim.csv")

#ETL "sales_type_dim"

In [ ]:
#Reading the dataset
sales_type_dim_df = pd.read_csv(directory_salh)
etl_sales_type_dim_df = sales_type_dim_df

In [ ]:
#Removing unused columns
etl_sales_type_dim_df.drop(['TRNO', 'CUSTCODE','TRDATE', 'TOTAL', 'UPDDATE', 'UPDTIME'], axis=1, inplace=True)

In [ ]:
#Changing the column names to match the star schema
etl_sales_type_dim_df.rename(columns = {"STYPE":"sales_type_name"}, inplace=True)

In [ ]:
#Dropping duplicate items from the dataframe
etl_sales_type_dim_df.drop_duplicates(inplace = True)

In [ ]:
#Resetting the dataframe index because the old dataframe index is not in order
etl_sales_type_dim_df.reset_index(drop=True, inplace=True)

In [ ]:
#Creating surrogate key for the "sales_type_dim" table

#Creating an empty dataframe with a column named "sales_type_id"
surrogate_key_sales_type_dim_df = pd.DataFrame(columns = ["sales_type_id"])

#Creating a surrogate key and then load the surrogate key into the empty dataframe that was previously created
for i in range(1, len(etl_sales_type_dim_df)+1, 1):
  surrogate_key_sales_type_dim_df = surrogate_key_sales_type_dim_df.append({'sales_type_id' : i}, ignore_index = True)

#Merging the surrogate key into the "sales_type_dim" dataframe 
etl_sales_type_dim_df["sales_type_id"] = surrogate_key_sales_type_dim_df

In [ ]:
#Sorting the columns to match the star schema
etl_sales_type_dim_df = etl_sales_type_dim_df[["sales_type_id","sales_type_name"]]

In [ ]:
#Changing the "sales_type_name" from "T" to "Tunai" and from "K" to "Kredit"
etl_sales_type_dim_df.loc[etl_sales_type_dim_df["sales_type_name"] == "T", "sales_type_name"] = "Tunai"
etl_sales_type_dim_df.loc[etl_sales_type_dim_df["sales_type_name"] == "K", "sales_type_name"] = "Kredit"

In [ ]:
#Converting the dataframe to CSV
etl_sales_type_dim_df.to_csv("sales_type_dim.csv", index=False)
files.download("sales_type_dim.csv")

#ETL "sales_approval_dim"

In [ ]:
#Reading the dataset
sales_approval_dim_df = pd.read_csv(directory_sald)
etl_sales_approval_dim_df = sales_approval_dim_df

In [ ]:
#Removing unused columns
etl_sales_approval_dim_df.drop(['SALDCODE', 'TRNO','ITEMNO', 'LINENO', 'UPRICE', 'UCOST', 'QTY', 'QTYRET', 'PREDICTQTY', 'PREDICTRET', 'DISCAMT', 'AMOUNT', 'UPDDATE', 'UPDTIME'], axis=1, inplace=True)

In [ ]:
#Changing the column names to match the star schema
etl_sales_approval_dim_df.rename(columns = {"APPROVED":"sales_approval_status"}, inplace=True)

In [ ]:
#Dropping duplicate items from the dataframe
etl_sales_approval_dim_df.drop_duplicates(inplace = True)

In [ ]:
#Resetting the dataframe index because the old dataframe index is not in order
etl_sales_approval_dim_df.reset_index(drop=True, inplace=True)

In [ ]:
#Adding a new "sales_approval_status" called "False" into the "sales_approval_dim" dataframe
approval_false = pd.DataFrame({"sales_approval_status":["False"]})
etl_sales_approval_dim_df = etl_sales_approval_dim_df.append(approval_false, ignore_index = True)

In [ ]:
#Creating surrogate key for the "sales_approval_dim" table

#Creating an empty dataframe with a column named "sales_approval_id"
surrogate_key_sales_approval_dim_df = pd.DataFrame(columns = ["sales_approval_id"])

#Creating a surrogate key and then load the surrogate key into the empty dataframe that was previously created
for i in range(1, len(etl_sales_approval_dim_df)+1, 1):
  surrogate_key_sales_approval_dim_df = surrogate_key_sales_approval_dim_df.append({'sales_approval_id' : i}, ignore_index = True)

#Merging the surrogate key into the "sales_approval_dim" dataframe  
etl_sales_approval_dim_df["sales_approval_id"] = surrogate_key_sales_approval_dim_df

In [ ]:
#Sorting the columns to match the star schema
etl_sales_approval_dim_df = etl_sales_approval_dim_df[["sales_approval_id","sales_approval_status"]]

In [ ]:
#Changing the "sales_approval_status" from "True" to "Disetujui" and from "False" to "Tidak Disetujui"
etl_sales_approval_dim_df.loc[etl_sales_approval_dim_df["sales_approval_status"] == True, "sales_approval_status"] = "Disetujui"
etl_sales_approval_dim_df.loc[etl_sales_approval_dim_df["sales_approval_status"] == "False", "sales_approval_status"] = "Tidak Disetujui"

In [ ]:
#Converting the dataframe to CSV
etl_sales_approval_dim_df.to_csv("sales_approval_dim.csv", index=False)
files.download("sales_approval_dim.csv")

#ETL "rfm_segmentation_dim"

In [ ]:
#Creating the "rfm_segmentation_dim" table and data 
data = [[0, "Low Spender"], [1, "Lost Customer"], [2, "Ex Best Customer"],[3, "Best Customer"], [4, "Need Reactivation"], [5, "Ex Low Spender"]]

rfm_segmentation_df = pd.DataFrame(data, columns=['rfm_segmentation_id', 'customer_group_name'])

In [ ]:
#Converting the dataframe to CSV
rfm_segmentation_df.to_csv("rfm_segmentation_dim.csv", index=False)
files.download("rfm_segmentation_dim.csv")

#ETL "sales_detail_fact"

##Perform JOIN to All Transactional Database Tables

In [ ]:
#Reading the dataset
cust_table_df = pd.read_csv(directory_cust, ";")
salh_table_df = pd.read_csv(directory_salh)
sald_table_df = pd.read_csv(directory_sald)
product_dim_df = pd.read_csv(directory_inv)
area_dim_df = pd.read_csv(directory_area)
sales_person_dim_df = pd.read_csv(directory_salesperson)

In [ ]:
#JOIN the "SALD" table with the "SALH" table
cust_table_df.rename(columns={"CODE": "CUSTCODE"}, inplace=True)

sald_join_salh_df = sald_table_df.merge(salh_table_df,
                    on=['TRNO'],
                    how="inner"
                    )

In [ ]:
#JOIN the "SALD, SALH" table with the "CUST" table
sald_join_salh_join_cust_df = sald_join_salh_df.merge(cust_table_df,
                    on=['CUSTCODE'],
                    how="inner"
                    )

In [ ]:
#JOIN the "SALD, SALH, CUST" table with the "INV" table
inv_to_cust_df = sald_join_salh_join_cust_df.merge(product_dim_df,
                    on=['ITEMNO'],
                    how="inner"
                    )

#Removing unused columns
inv_to_cust_df.drop(['UPDDATE_x', 'UPDTIME_x','UPDDATE_y', 'UPDTIME_y','UCOST_y','WEIGHT','PHONE','ADDR1','ADDR2','ADDR3', 'SPRICE'], axis=1, inplace=True)

In [ ]:
#JOIN the "SALD, SALH, CUST, INV" table with the "AREA" table
area_dim_df.rename(columns={"CODE": "AREACD"}, inplace=True)

inv_to_area_df = inv_to_cust_df.merge(area_dim_df,
                    on=['AREACD'],
                    how="inner"
                    )

#Removing unused columns
inv_to_area_df.drop(['UPDDATE', 'UPDTIME'], axis=1, inplace=True)

In [ ]:
#JOIN the "SALD, SALH, CUST, INV, AREA" table with the "SALPERSON" table
inv_to_area_df.rename(columns={"SALPERSON": "SALPCODE"}, inplace=True)

inv_to_salp_df = inv_to_area_df.merge(sales_person_dim_df,
                    on=['SALPCODE'],
                    how="inner"
                    )

#Removing unused columns
inv_to_salp_df.drop(['UPDDATE', 'UPDTIME'], axis=1, inplace=True)

##Perform JOIN to All Data Warehouse Tables

In [ ]:
#JOIN the transactional database dataframe with the "product_dim" dataframe to get the "product_id"
inv_to_salp_df.rename(columns={"ITEMNAME": "product_name"}, inplace=True)

df_after_join_product_dim = inv_to_salp_df.merge(etl_product_dim_df,
                    on=['product_name'],
                    how="inner"
                    )

In [ ]:
#JOIN the transactional database dataframe with the "customer_dim" dataframe to get the "customer_id"
df_after_join_product_dim.rename(columns={"CUSTNAME": "customer_name"}, inplace=True)

df_after_join_customer_dim = df_after_join_product_dim.merge(etl_customer_dim_df,
                    on=['customer_name'],
                    how="inner"
                    )

In [ ]:
#JOIN the transactional database dataframe with the "date_dim" dataframe to get "date_id"
df_after_join_customer_dim.rename(columns={"TRDATE": "transaction_date"}, inplace=True)

df_after_join_customer_dim['transaction_date'] = pd.to_datetime(df_after_join_customer_dim['transaction_date'])

df_after_join_date_dim = df_after_join_customer_dim.merge(etl_date_dim_df,
                    on=['transaction_date'],
                    how="inner"
                    )

In [ ]:
#JOIN the transactional database dataframe with the "sales_person_dim" dataframe to get the "sales_person_id"
df_after_join_date_dim.rename(columns={"SALPNAME": "sales_person_name"}, inplace=True)

df_after_join_sales_person_dim = df_after_join_date_dim.merge(etl_sales_person_dim_df,
                    on=['sales_person_name'],
                    how="inner"
                    )

In [ ]:
#JOIN the transactional database dataframe with the "area_dim" dataframe to get the "area_id"
df_after_join_sales_person_dim.rename(columns={"DESC": "area_name"}, inplace=True)

df_after_join_area_dim = df_after_join_sales_person_dim.merge(etl_area_dim_df,
                    on=['area_name'],
                    how="inner"
                    )

In [ ]:
#JOIN the transactional database dataframe with the "sales_type_dim" dataframe to get "sales_type_id"
df_after_join_area_dim.loc[df_after_join_area_dim["STYPE"] == "T", "STYPE"] = "Tunai"
df_after_join_area_dim.loc[df_after_join_area_dim["STYPE"] == "K", "STYPE"] = "Kredit"

df_after_join_area_dim.rename(columns={"STYPE": "sales_type_name"}, inplace=True)

df_after_join_sales_type_dim = df_after_join_area_dim.merge(etl_sales_type_dim_df,
                    on=['sales_type_name'],
                    how="inner"
                    )

In [ ]:
#JOIN the transactional database dataframe with the "sales_approval_dim" dataframe to get the "sales_approval_id"
df_after_join_sales_type_dim.loc[df_after_join_sales_type_dim["APPROVED"] == True, "APPROVED"] = "Disetujui"
df_after_join_sales_type_dim.loc[df_after_join_sales_type_dim["APPROVED"] == False, "APPROVED"] = "Tidak Disetujui"

df_after_join_sales_type_dim.rename(columns={"APPROVED": "sales_approval_status"}, inplace=True)

df_after_join_sales_approval_dim = df_after_join_sales_type_dim.merge(etl_sales_approval_dim_df,
                    on=['sales_approval_status'],
                    how="inner"
                    )

In [ ]:
#JOIN the transactional database dataframe with the "rfm_segmentation_dim" dataframe to get the recency, frequency, and monetary value

#JOIN the clustering result with the RFM category dataframe
rfm_per_customer_df = pd.read_csv(directory_rfm, ",")

rfm_per_customer_df.drop(['R', 'F','M'], axis=1, inplace=True)

rfm_per_customer_df.rename(columns={"Cluster": "rfm_segmentation_id"}, inplace=True)

df_rfm_joined = rfm_per_customer_df.merge(rfm_segmentation_df,
                    on=['rfm_segmentation_id'],
                    how="inner"
                    )

df_rfm_joined.rename(columns={"customer_id": "customer_name"}, inplace=True)

#JOIN the RFM table with the "sales_detail_fact" dataframe
df_after_join_rfm_dim = df_after_join_sales_approval_dim.merge(df_rfm_joined,
                    on=['customer_name'],
                    how="inner"
                    )

##Reorganizing The "sales_detail_fact" Dataframe To Match The Star Schema

In [ ]:
etl_sales_detail_fact = df_after_join_rfm_dim

#Removing unused columns
etl_sales_detail_fact.drop(['product_name', 'INACTIVE', 'area_name','sales_person_name', 'DOB','GENDER','SALARY','SALTYPE', 'LINENO','sales_approval_status','customer_status','customer_group_name','year','quarter','month','week', 'day','latitude','longitude', 'transaction_date', 'sales_type_name','AREACD', 'CUSTCODE', 'customer_name'], axis=1, inplace=True)

#Changing the column names to match the star schema
etl_sales_detail_fact.rename(columns={"SALDCODE": "old_sales_detail_id", "TRNO": "old_sales_header_id", "ITEMNO":"product_id", "UPRICE": "price", "UCOST_x": "production_cost", "QTY":"quantity", "QTYRET":"return_quantity", "PREDICTQTY":"predicted_buy_quantity", "PREDICTRET": "predicted_return_quantity", "DISCAMT": "discount_amount", "AMOUNT":"sub_total", "TOTAL":"grand_total", "ARBAL":"accounts_receivable", "SALPCODE": "sales_person_id"}, inplace=True)

fact_table_copy = etl_sales_detail_fact

#Removing duplicate columns
no_duplicate_fact  = fact_table_copy.loc[:,~fact_table_copy.columns.duplicated()]

#Adding the "actual_demand" and "predicted_demand" columns
no_duplicate_fact['actual_demand'] = no_duplicate_fact['quantity'] - no_duplicate_fact['return_quantity']
no_duplicate_fact['predicted_demand'] = no_duplicate_fact['predicted_buy_quantity'] - no_duplicate_fact['predicted_return_quantity']

#Sorting the columns to match the star schema
no_duplicate_fact_reordered = no_duplicate_fact[['old_sales_header_id', 'old_sales_detail_id', 'product_id', 'customer_id', 'date_id', 'sales_person_id', 'area_id', 'sales_type_id', 'sales_approval_id', 'rfm_segmentation_id', 'price', 'production_cost', 'accounts_receivable', 'predicted_buy_quantity', 'predicted_return_quantity', 'predicted_demand', 'quantity',  'return_quantity', 'actual_demand', 'discount_amount', 'sub_total', 'grand_total', 'recency', 'frequency', 'monetary_value']]

##Creating Composite Key For The "sales_detail_fact" Table

In [ ]:
composite_key_sales_detail_fact_df = no_duplicate_fact_reordered['old_sales_header_id'] + '-' +  no_duplicate_fact_reordered['old_sales_detail_id'] + '-' + no_duplicate_fact_reordered['product_id'].map(str) + '-' +  no_duplicate_fact_reordered['customer_id'].map(str) + '-' + no_duplicate_fact_reordered['date_id'].map(str) + '-' +  no_duplicate_fact_reordered['sales_person_id'].map(str) + '-' + no_duplicate_fact_reordered['area_id'].map(str) + '-' +  no_duplicate_fact_reordered['sales_type_id'].map(str)  + '-' +  no_duplicate_fact_reordered['sales_approval_id'].map(str) + '-' + no_duplicate_fact_reordered['rfm_segmentation_id'].map(str)

#Combining "sales_detail_fact" dataframe with the composite key dataframe
no_duplicate_fact_reordered = no_duplicate_fact_reordered.merge(composite_key_sales_detail_fact_df.rename('sales_detail_fact_id'), left_index=True, right_index=True)

#Sorting the columns to match the star schema
no_duplicate_fact_reordered = no_duplicate_fact_reordered[["sales_detail_fact_id",'old_sales_header_id', 'old_sales_detail_id', 'product_id', 'customer_id', 'date_id', 'sales_person_id', 'area_id', 'sales_type_id', 'sales_approval_id', 'rfm_segmentation_id', 'price', 'production_cost', 'accounts_receivable', 'predicted_buy_quantity', 'predicted_return_quantity', 'predicted_demand', 'quantity',  'return_quantity', 'actual_demand', 'discount_amount', 'sub_total', 'grand_total', 'recency', 'frequency', 'monetary_value']]

##Converting The "sales_detail_fact" Dataframe to CSV

In [ ]:
no_duplicate_fact_reordered.to_csv("sales_detail_fact.csv", index=False)
files.download("sales_detail_fact.csv")